<h2>SPYPredictor</h2>
Final Project Shuzo Katayama, 11 Deecember 2020

15 November 2020: 

Progress for the Prototype:

Data:

SPY Prices: https://finance.yahoo.com/quote/SPY/history?period1=728265600&period2=1604966400&interval=1mo&filter=history&frequency=1mo&includeAdjustedClose=true \
GDP: https://fred.stlouisfed.org/series/GDP \
Unemployment: https://fred.stlouisfed.org/series/UNRATE \
Federal Funds Rate: https://fred.stlouisfed.org/series/FEDFUNDS 

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

Importing the data:\
All the data will come initially in a DataFrame with the name of the data followed by 'df'

In [2]:
# Daily prices of SPY
SPYdf = pd.read_csv('SPYDaily.csv')
SPYdf

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-01-29,43.968750,43.968750,43.750000,43.937500,26.079659,1003200
1,1993-02-01,43.968750,44.250000,43.968750,44.250000,26.265144,480500
2,1993-02-02,44.218750,44.375000,44.125000,44.343750,26.320782,201300
3,1993-02-03,44.406250,44.843750,44.375000,44.812500,26.599014,529400
4,1993-02-04,44.968750,45.093750,44.468750,45.000000,26.710312,531500
...,...,...,...,...,...,...,...
6992,2020-11-03,333.690002,338.250000,330.290009,336.029999,336.029999,93294200
6993,2020-11-04,340.859985,347.940002,339.589996,343.540009,343.540009,126959700
6994,2020-11-05,349.239990,352.190002,348.859985,350.239990,350.239990,82039700
6995,2020-11-06,349.929993,351.510010,347.649994,350.160004,350.160004,74973000


In [3]:
GDPdf = pd.read_csv('GDPQuarterly.csv')
Unemploymentdf = pd.read_csv('UnemploymentMonthly.csv')
Unemploymentdf['UNRATE'].astype('float')
FedFundspd = pd.read_csv('Fedfunds.csv')

Splitting Data into X and y

In [4]:
# Training targets are the closing prices of SPY
y = SPYdf.to_numpy()
y = y[:, 4]
      
# Initialise Xdf as initially having 4 columns and the number of rows in SPYdf
c = 4
r = len(SPYdf)

a = SPYdf.to_numpy()
X = np.empty((r,c))

# Keep a separate array of the dates
Xdate = SPYdf.to_numpy()
Xdate = Xdate[:,0]

# Copy into column 0 in X, the opening price from a
counter = 0
for item in a:
    X[counter, 0] = item[1]
    counter = counter+1
    
# Copy in to column 1, the GDP
# GDP is calculated quarterly, and SPY is daily; writing GDP value every day in the quarter in X
g = GDPdf.to_numpy()
counter = r-1
gdpcounter = (len(g)-1)
for item in X:
    if g[gdpcounter,0] == Xdate[counter]:
        gdpcounter = gdpcounter - 1
    
    item[1] = g[gdpcounter, 1]
    counter = counter - 1
    
# Copy into column 2, the Unemployment
# Unemployment is calculated monthly; writing Unemployment every day in the month in X
u = Unemploymentdf.to_numpy()
counter = r-1
ucounter = (len(u)-1)
for item in X:
    if u[ucounter, 0] == Xdate[counter]:
        ucounter = ucounter - 1
        
    item[2] = u[ucounter, 1]
    counter = counter - 1

np.set_printoptions(suppress=True)

# Copy into column 3, the Federal Funds Rate
# Federal Funds Rate is calculated monthly;
f = FedFundspd.to_numpy()
counter = r-1
fcounter = (len(f)-1)
for item in X:
    if f[fcounter, 0] == Xdate[counter]:
        fcounter = fcounter - 1
        
    item[3] = f[fcounter, 1]
    counter = counter - 1

Splitting data into training and testing sets using train_test_split

In [5]:
# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

Model RF0: Training the Random Forest Regressor, from sci-kit learn

In [6]:
from sklearn.ensemble import RandomForestRegressor
est = RandomForestRegressor(n_estimators=1000, criterion='mse', random_state=1, n_jobs=-1)
est.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=1)

In [7]:
y_train_pred = est.predict(X_train)
y_test_pred = est.predict(X_test)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 0.521, test: 2.904
R^2 train: 1.000, test: 0.999


Model RF1: Training the Random Forest Regressor with standardised data

In [8]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

sc_y = StandardScaler()
sc.fit(y_train[:, np.newaxis])
y_train_std = sc.transform(y_train[:, np.newaxis]).flatten()
y_test_std = sc.transform(y_test[:, np.newaxis]).flatten() 

In [9]:
est = RandomForestRegressor(n_estimators=10000, criterion='mse', random_state=1, n_jobs=-1)
est.fit(X_train_std, y_train_std)

RandomForestRegressor(n_estimators=10000, n_jobs=-1, random_state=1)

In [10]:
y_train_pred = est.predict(X_train)
y_test_pred = est.predict(X_test)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train_std, y_train_pred),
        mean_squared_error(y_test_std, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train_std, y_train_pred),
        r2_score(y_test_std, y_test_pred)))

MSE train: 10.117, test: 10.060
R^2 train: -9.117, test: -9.545


After standardising the data, the model performs horribly. This is expected because the data that is inputted is only updated monthly or quarterly, and it is probably very difficult for the model to learn anything meaningful from this data. I asked it to predict daily fluctuations with monthly / quarterly data. From here, I will try to include sentiment analysis data of newspaper headlines in order to fine tune the result. (See SPYPredictor Sentiment Analysis.ipynb for more details)

Adding VADER Sentiment Analysis scores to the SPYPredictor

In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
headlines = pd.read_csv('abcnews-date-text.csv', encoding = "ISO-8859-1")
headlines

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
...,...,...
1186013,20191231,vision of flames approaching corryong in victoria
1186014,20191231,wa police and government backflip on drug amne...
1186015,20191231,we have fears for their safety: victorian premier
1186016,20191231,when do the 20s start


In [12]:
a = headlines.to_numpy()

In [13]:
# Initialise the array with average scores as having 6151 (days) rows, and 2 columns
r = 6151
c = 2
total_scores = np.empty((r, c))

In [14]:
# Loop through large headlines dataset. Get a compound score for each headline
# After each headline for a particular date is referenced, add it to the sum for that day
# Once its the next day, average the scores, and add the result to total_scores

sid = SentimentIntensityAnalyzer()
publish_date = a[0, 0]
counter = 1
score_sum = 0
total_counter = 0

for item in a:
    if item[0] == publish_date:
        scores = sid.polarity_scores(item[1])
        compound_score = scores['compound']
        score_sum = score_sum + compound_score
        counter = counter+1
    else:
        total_scores[total_counter, 0] = publish_date
        total_scores[total_counter, 1] = (score_sum/counter)
        
        publish_date = item[0]
        counter = 1
        total_counter = total_counter+1
        score_sum = 0

total_scores

array([[20030219.        ,       -0.10707286],
       [20030220.        ,       -0.1079212 ],
       [20030221.        ,       -0.0993488 ],
       ...,
       [20191228.        ,       -0.00996333],
       [20191229.        ,       -0.09816744],
       [20191230.        ,       -0.17128913]])

In [15]:
# Recreate X and y but within the range of the scores available for total_scores
# Initialise the array with average scores as having 6151 (days) rows, and 6 columns
r = 6151
c = 5
X_sentiment = np.empty((r, c))
y_sentiment = np.empty((r, c))
a = SPYdf.to_numpy()

In [16]:
# Keep a separate array of the dates within total_scores
XS_date = total_scores[:, 0]

# Type for XS_date dates is wrong, so use pandas to convert it to datetime
s = pd.DataFrame(XS_date)
s[0] = pd.to_datetime(s[0], format='%Y%m%d') 
XS_date = s.to_numpy()

# Type for Xdate dates is wrong, so use pandas to convert it to datetime
s = pd.DataFrame(Xdate)
s[0] = pd.to_datetime(s[0], format='%Y-%m-%d') 
Xdate = s.to_numpy()

# Reference: 'How to Convert Integer to Datetime in Pandas DataFrame?', GeeksforGeeks, 2020
# https://www.geeksforgeeks.org/how-to-convert-integer-to-datetime-in-pandas-dataframe/

In [17]:
# Loop through the dates, and if the dates are equal, plug in the values from the previous X and the 
# sentiment scores
XScounter = 0
Xcounter = 0

# First get XS and X date up to the same date
while XS_date[XScounter] != Xdate[Xcounter]:
    Xcounter = Xcounter+1

for x in range(0, 3618):
    
    # if XScounter < 6151 and Xcounter < 6997:
    while XS_date[XScounter] < Xdate[Xcounter]:
        XScounter = XScounter+1

    if XS_date[XScounter] == Xdate[Xcounter]:
        
        X_sentiment[XScounter, 0] = X[Xcounter, 0]
        X_sentiment[XScounter, 1] = X[Xcounter, 1]
        X_sentiment[XScounter, 2] = X[Xcounter, 2]
        X_sentiment[XScounter, 3] = X[Xcounter, 3]
        X_sentiment[XScounter, 4] = total_scores[XScounter, 1]
        y_sentiment[XScounter] = y[Xcounter]
        
    XScounter = XScounter+1
    Xcounter = Xcounter+1


In [18]:
# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

Training the Random Forest Regressor, with Sentiment Analysis Data (standardised)

In [19]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

sc_y = StandardScaler()
sc.fit(y_train[:, np.newaxis])
y_train_std = sc.transform(y_train[:, np.newaxis]).flatten()
y_test_std = sc.transform(y_test[:, np.newaxis]).flatten() 

In [20]:
est = RandomForestRegressor(n_estimators=1000, criterion='mse', random_state=1, n_jobs=-1)
est.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=1)

In [21]:
y_train_pred = est.predict(X_train)
y_test_pred = est.predict(X_test)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 0.521, test: 2.904
R^2 train: 1.000, test: 0.999


The final model with the Sentiment Analysis data performed surprisingly well. The R<sup>2</sup> of the test model was at 0.999 . This indicates that the model was able to predict the movement of SPY with the sentiment analysis data very well. However, I still question if there were flaws in my method, because these results seem almost too good. An inspection into y_test and y_test_pred, shows that each of the predictions are very close.

In [22]:
y_test

array([137.880005, 107.529999, 125.129997, ..., 285.670013, 208.080002,
       144.929993], dtype=object)

In [23]:
y_test_pred

array([137.08798356, 108.00058627, 126.09699222, ..., 287.23091665,
       210.39007631, 146.74732295])

In conclusion, the prediction of the movement of the entire market is a very tricky process. Because the performance of the market is something that is the subject of endless debate, and the factors of the market performance is also something that is hotly contested, it is, of course, very difficult to see which factors affect the performance of SPY. However, in this project, it seems that I am able to conclude that the news and its sentiment can be an indicator for how the market moves. 

In the future, the sentiment analysis can be improved to yield better results. In this case, I used VADER because the presence of pre-labelled data was difficult to find, but with enough time, this data can probably be created. There is also room to grow in terms of sentiment analysis learning. 

Works Cited: 

'How to Convert Integer to Datetime in Pandas DataFrame?', GeeksforGeeks, 2020
https://www.geeksforgeeks.org/how-to-convert-integer-to-datetime-in-pandas-dataframe/

"6.2. Feature extraction", Scikit-Learn Documentation
https://scikit-learn.org/stable/modules/feature_extraction.html

"VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text", by C.J. Hutto Eric Gilbert, Georgia Institute of Technology, 2014 http://eegilbert.org/papers/icwsm14.vader.hutto.pdf

"Python | Sentiment Analysis using VADER", Geeksforgeeks.org, 2019 https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/